In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import fitsio
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, project_to_multipoles, project_to_wp, utils, setup_logging
from scipy.optimize import curve_fit
from LSS.common_tools import mknz
from astropy.table import Table
import itertools

from dataloc import *
from groupcatalog import read_wp_file

# MAKE ALL PLOTS TEXT BIGGER
plt.rcParams.update({'font.size': 15})
# But legend a bit smaller
plt.rcParams.update({'legend.fontsize': 12})
# Set DPI up a bit
plt.rcParams.update({'figure.dpi': 150})


This notebook creates takes an LSSCatalog and cuts the data and randoms to a little patch to be used for rapid validation.

In [ ]:
dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER, 'Y1', 'LSS', 'iron', 'LSScats', 'v1.5pip')
outdir = os.path.join(dir, 'mini')
print(f'Using directory {dir} for data')
print(f'Using directory {outdir} for output')

In [ ]:
RA_MAX = 175
RA_MIN = 160
DEC_MAX = 3
DEC_MIN = -7
# Despite being a 150 deg sq cut, the randoms measure it to be ~141.6 deg sq.
def reduce_table(tbl):
    ol = len(tbl)
    sel = np.ones(len(tbl), dtype=bool)
    sel &= (tbl['RA'] < RA_MAX)
    sel &= (tbl['RA'] > RA_MIN)
    sel &= (tbl['DEC'] < DEC_MAX)
    sel &= (tbl['DEC'] > DEC_MIN)
    reduced = tbl[sel]
    print(f'Reduced table from {ol} to {len(reduced)}')
    return reduced

In [ ]:
dpath = os.path.join(dir, f'BGS_BRIGHT_full_HPmapcut.dat.fits')
dtbl = fitsio.read(dpath)
dtbl = reduce_table(dtbl)
fitsio.write(os.path.join(outdir, 'BGS_BRIGHT_full_HPmapcut.dat.fits'), dtbl)
del dtbl

for j in np.arange(0, 18):
    rpath = os.path.join(dir, f'BGS_BRIGHT_{j}_full_HPmapcut.ran.fits')
    rtbl = fitsio.read(rpath)
    rtbl = reduce_table(rtbl)
    fitsio.write(os.path.join(outdir, f'BGS_BRIGHT_{j}_full_HPmapcut.ran.fits'), rtbl)
    del rtbl

In [ ]:
#for i in ['N', 'S']: # The area is only in NGC anyway
for i in ['N']:
    dpath = os.path.join(dir, f'BGS_BRIGHT_{i}GC_clustering.dat.fits')
    dtbl = fitsio.read(dpath)
    dtbl = reduce_table(dtbl)
    fitsio.write(os.path.join(outdir, f'BGS_BRIGHT_{i}GC_clustering.dat.fits'), dtbl)
    del dtbl

    for j in np.arange(0, 18):
        rpath = os.path.join(dir, f'BGS_BRIGHT_{i}GC_{j}_clustering.ran.fits')
        rtbl = fitsio.read(rpath)
        rtbl = reduce_table(rtbl)
        fitsio.write(os.path.join(outdir, f'BGS_BRIGHT_{i}GC_{j}_clustering.ran.fits'), rtbl)
        del rtbl